In [1]:
from pathlib import Path
import shutil
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer
from datasets import load_dataset
import evaluate
import numpy as np
import pandas as pd
import os
metric = evaluate.load("accuracy")


In [2]:
# "bert-base-chinese"  # "distilbert-base-uncased"
MODEL_NAME_OR_PATH = "hfl/chinese-roberta-wwm-ext"
NUM_LABELS = 2
MAX_LENGTH = 64

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME_OR_PATH, num_labels=NUM_LABELS)


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH)


Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model che

In [3]:
text_dataset = load_dataset('csv', data_files={
                            'train': ['data_all/data/train_data.csv'], 'test': ['data_all/data/test_data.csv']})


def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=MAX_LENGTH)


tokenized_text = text_dataset.map(preprocess_function, batched=True)


data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer, max_length=MAX_LENGTH)


Using custom data configuration default-28e3bfa34012c0da


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

c:\Users\yuanz\anaconda3\envs\mynet2\lib\site-packages\datasets\download\streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/yuanz/.cache/huggingface/datasets/csv/default-28e3bfa34012c0da/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


c:\Users\yuanz\anaconda3\envs\mynet2\lib\site-packages\datasets\download\streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [4]:


LOGGGING_DIR = "loggging_dir"
MODEL_DIR = "model_result"
shutil.rmtree(LOGGGING_DIR, ignore_errors=True)
training_args = TrainingArguments(
    output_dir=MODEL_DIR,
    overwrite_output_dir=True,
    logging_dir=LOGGGING_DIR,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    do_eval=True,
    evaluation_strategy="steps",
    eval_accumulation_steps=50,
    eval_steps=50,
    logging_steps=50,
    save_steps=100,
    num_train_epochs=4,
    weight_decay=0.01,
    save_total_limit=3,
    jit_mode_eval=True,
    fp16=True,
    fp16_opt_level='O3',
    load_best_model_at_end=True,

)


In [5]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_text["train"],
    eval_dataset=tokenized_text["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()


Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
c:\Users\yuanz\anaconda3\envs\mynet2\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6988
  Num Epochs = 4
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 876
  Number of trainable parameters = 102269186


  0%|          | 0/876 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
c:\Users\yuanz\anaconda3\envs\mynet2\lib\site-packages\transformers\tokenization_utils_base.py:2357: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.


{'loss': 0.4357, 'learning_rate': 1.8881278538812787e-05, 'epoch': 0.23}


failed to use PyTorch jit mode due to: The size of tensor a (64) must match the size of tensor b (32) at non-singleton dimension 1.
***** Running Evaluation *****
  Num examples = 776
  Batch size = 32


...

  0%|          | 0/25 [00:00<?, ?it/s]

{'eval_loss': 0.08777593821287155, 'eval_jit_compilation_time': 0.1767, 'eval_accuracy': 0.9677835051546392, 'eval_runtime': 0.443, 'eval_samples_per_second': 1751.876, 'eval_steps_per_second': 56.439, 'epoch': 3.88}


c:\Users\yuanz\anaconda3\envs\mynet2\lib\site-packages\transformers\tokenization_utils_base.py:2357: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from model_result\checkpoint-800 (score: 0.09813018888235092).


{'train_runtime': 86.8391, 'train_samples_per_second': 321.883, 'train_steps_per_second': 10.088, 'train_loss': 0.18248712390525157, 'epoch': 4.0}


TrainOutput(global_step=876, training_loss=0.18248712390525157, metrics={'train_runtime': 86.8391, 'train_samples_per_second': 321.883, 'train_steps_per_second': 10.088, 'train_loss': 0.18248712390525157, 'epoch': 4.0})